In [ ]:
# =====================================
# INSTALL MISSING DEPENDENCIES (SAFE)
# =====================================
import sys
!{sys.executable} -m pip install seaborn


In [ ]:
# =====================================
# INSTALL REQUIRED EXCEL DEPENDENCY
# =====================================
import sys
!{sys.executable} -m pip install openpyxl


In [ ]:
import sys
!{sys.executable} -m pip install matminer pymatgen


In [ ]:
import sys
!{sys.executable} -m pip install xgboost lightgbm


In [ ]:
# ================================
# CORE SCIENTIFIC STACK
# ================================
import numpy as np
import pandas as pd

# ================================
# VISUALIZATION
# ================================
import matplotlib.pyplot as plt
import seaborn as sns

# ================================
# FILE & PATH MANAGEMENT
# ================================
from pathlib import Path
import os

# ================================
# WARNINGS CONTROL
# ================================
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# ================================
# GLOBAL PLOT SETTINGS
# ================================
plt.rcParams.update({
    "figure.figsize": (7, 5),
    "figure.dpi": 120,
    "savefig.dpi": 300,
    "font.size": 11,
    "axes.labelsize": 12,
    "axes.titlesize": 13,
    "legend.fontsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "lines.linewidth": 2,
})
sns.set_style("whitegrid")


In [ ]:
# ================================
# PROJECT ROOT
# ================================
PROJECT_ROOT = Path(".").resolve()

# ================================
# DATA DIRECTORIES
# ================================
DATA_RAW_DIR = PROJECT_ROOT / "data" / "raw"
DATA_CLEAN_DIR = PROJECT_ROOT / "data" / "clean"
DATA_INTERIM_DIR = PROJECT_ROOT / "data" / "intermediate"

# ================================
# RESULTS DIRECTORIES
# ================================
RESULTS_DIR = PROJECT_ROOT / "results"
TABLES_DIR = RESULTS_DIR / "tables"
FIGURES_DIR = RESULTS_DIR / "figures"

# ================================
# FIGURE SUBDIRECTORIES
# ================================
FIG_PNG_DIR = FIGURES_DIR / "png"
FIG_PDF_DIR = FIGURES_DIR / "pdf"

# ================================
# CREATE ALL DIRECTORIES
# ================================
for d in [
    DATA_RAW_DIR, DATA_CLEAN_DIR, DATA_INTERIM_DIR,
    RESULTS_DIR, TABLES_DIR,
    FIGURES_DIR, FIG_PNG_DIR, FIG_PDF_DIR
]:
    d.mkdir(parents=True, exist_ok=True)

print("✅ Project directory structure ready.")



In [ ]:
# ================================
# STANDARD FILE NAMES (DO NOT CHANGE)
# ================================
RAW_DATA_FILE = DATA_RAW_DIR / "Curie_Data_Raw.xlsx"
CLEAN_DATA_FILE = DATA_CLEAN_DIR / "Curie_Data_Cleaned.csv"

print("Raw data file  :", RAW_DATA_FILE)
print("Clean data file:", CLEAN_DATA_FILE)

In [ ]:
# ================================
# UNIFIED FIGURE SAVE FUNCTION
# ================================
def save_figure(fig, name, tight=True):
    """
    Save figure in both PNG and PDF formats.
    
    Parameters:
    - fig: matplotlib figure object
    - name: file name WITHOUT extension
    - tight: apply tight_layout before saving
    """
    if tight:
        fig.tight_layout()
    
    png_path = FIG_PNG_DIR / f"{name}.png"
    pdf_path = FIG_PDF_DIR / f"{name}.pdf"
    
    fig.savefig(png_path, bbox_inches="tight")
    fig.savefig(pdf_path, bbox_inches="tight")
    
    print(f"Saved: {png_path}")
    print(f"Saved: {pdf_path}")


In [ ]:
print("Python version check OK")
print("pandas:", pd.__version__)
print("numpy :", np.__version__)


In [ ]:
# ============================================================
# CELL 1: DATA LOADING & STRICT CLEANING (nat DROPPED)
# ============================================================

# -------------------------------
# 1. Load raw dataset
# -------------------------------
df_raw = pd.read_excel(RAW_DATA_FILE)

print("=== RAW DATA (AS LOADED) ===")
print("Shape:", df_raw.shape)
print("Columns:", df_raw.columns.tolist())

# Save raw snapshot (for traceability)
raw_snapshot_path = DATA_INTERIM_DIR / "Curie_Data_Raw_snapshot.csv"
df_raw.to_csv(raw_snapshot_path, index=False)

print("Raw snapshot saved to:", raw_snapshot_path)


# -------------------------------
# 2. Drop 'nat' column explicitly
# -------------------------------
if "nat" in df_raw.columns:
    df_raw = df_raw.drop(columns=["nat"])
    print("\n'nat' column dropped.")
else:
    print("\n'nat' column not present.")

print("Columns after dropping 'nat':", df_raw.columns.tolist())


# -------------------------------
# 3. Remove rows with ANY missing values
#    (now only checks Material_Name & Curie)
# -------------------------------
df_no_missing = df_raw.dropna(how="any")

print("\n=== AFTER DROPPING MISSING VALUES ===")
print("Shape:", df_no_missing.shape)
print("Rows removed (missing):", df_raw.shape[0] - df_no_missing.shape[0])


# -------------------------------
# 4. Remove duplicate rows
# -------------------------------
df_clean = df_no_missing.drop_duplicates()

print("\n=== AFTER DROPPING DUPLICATES ===")
print("Shape:", df_clean.shape)
print("Duplicate rows removed:", df_no_missing.shape[0] - df_clean.shape[0])


# -------------------------------
# 5. Reset index (critical for reproducibility)
# -------------------------------
df_clean = df_clean.reset_index(drop=True)


# -------------------------------
# 6. Save cleaned dataset
# -------------------------------
df_clean.to_csv(CLEAN_DATA_FILE, index=False)

print("\n=== CLEAN DATA SAVED ===")
print("Cleaned data file:", CLEAN_DATA_FILE)


# -------------------------------
# 7. Save cleaning log (Methods-ready)
# -------------------------------
cleaning_log = pd.DataFrame({
    "Stage": [
        "Raw dataset (with nat)",
        "After dropping nat column",
        "After removing missing values",
        "After removing duplicates"
    ],
    "Rows": [
        pd.read_excel(RAW_DATA_FILE).shape[0],
        df_raw.shape[0],
        df_no_missing.shape[0],
        df_clean.shape[0]
    ],
    "Columns": [
        3,                      # original columns
        df_raw.shape[1],
        df_no_missing.shape[1],
        df_clean.shape[1]
    ]
})

log_path = RESULTS_DIR / "data_cleaning_log.csv"
cleaning_log.to_csv(log_path, index=False)

print("\nCleaning log saved to:", log_path)


In [ ]:
# ============================================================
# CELL 2: SANITY CHECKS & Tc DISTRIBUTION
# ============================================================

# -------------------------------
# 1. Load cleaned dataset
# -------------------------------
df = pd.read_csv(CLEAN_DATA_FILE)

print("=== CLEAN DATASET LOADED ===")
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())

# -------------------------------
# 2. Basic sanity checks
# -------------------------------
print("\n=== SANITY CHECKS ===")
print("Missing values per column:")
print(df.isnull().sum())

print("\nUnique materials:", df["Material_Name"].nunique())

# -------------------------------
# 3. Basic Tc statistics
# -------------------------------
tc_stats = df["Curie"].describe()

print("\n=== Curie Temperature Statistics (K) ===")
print(tc_stats)

# Save statistics table
tc_stats_df = tc_stats.to_frame(name="Curie_Temperature_K")
stats_path = TABLES_DIR / "Curie_Tc_basic_statistics.csv"
tc_stats_df.to_csv(stats_path)

print("\nTc statistics table saved to:", stats_path)

# -------------------------------
# 4. Histogram: Tc distribution (linear scale)
# -------------------------------
fig1, ax1 = plt.subplots()

ax1.hist(df["Curie"], bins=60)
ax1.set_xlabel("Curie Temperature (K)")
ax1.set_ylabel("Count")
ax1.set_title("Distribution of Curie Temperatures (Linear Scale)")

save_figure(fig1, "Tc_distribution_linear")

plt.close(fig1)

# -------------------------------
# 5. Histogram: Tc distribution (log scale)
# -------------------------------
fig2, ax2 = plt.subplots()

ax2.hist(df["Curie"], bins=60)
ax2.set_xlabel("Curie Temperature (K)")
ax2.set_ylabel("Count (log scale)")
ax2.set_yscale("log")
ax2.set_title("Distribution of Curie Temperatures (Log Scale)")

save_figure(fig2, "Tc_distribution_log")

plt.close(fig2)

# -------------------------------
# 6. Optional: Boxplot (outlier visibility)
# -------------------------------
fig3, ax3 = plt.subplots()

ax3.boxplot(df["Curie"], vert=False)
ax3.set_xlabel("Curie Temperature (K)")
ax3.set_title("Boxplot of Curie Temperatures")

save_figure(fig3, "Tc_boxplot")

plt.close(fig3)

print("\nCELL 2 completed successfully.")


In [ ]:
# ============================================================
# CELL 3: CHEMISTRY-AWARE INSPECTION
# ============================================================

import re
from collections import Counter

# -------------------------------
# 1. Helper: parse elements from formula
# -------------------------------
def extract_elements(formula):
    """
    Extract element symbols from a chemical formula string.
    Example: 'Co2Fe0.75V0.25Ge' -> ['Co', 'Fe', 'V', 'Ge']
    """
    return re.findall(r"[A-Z][a-z]?", formula)

# -------------------------------
# 2. Extract elemental information
# -------------------------------
df["elements"] = df["Material_Name"].apply(extract_elements)
df["n_elements"] = df["elements"].apply(lambda x: len(set(x)))

# -------------------------------
# 3. Identify key magnetic elements
# -------------------------------
MAGNETIC_3D = {"Fe", "Co", "Ni", "Mn", "Cr", "V"}
RARE_EARTHS = {
    "La","Ce","Pr","Nd","Sm","Eu","Gd","Tb","Dy",
    "Ho","Er","Tm","Yb","Lu"
}

df["has_3d_TM"] = df["elements"].apply(
    lambda els: any(e in MAGNETIC_3D for e in els)
)

df["has_RE"] = df["elements"].apply(
    lambda els: any(e in RARE_EARTHS for e in els)
)

# -------------------------------
# 4. Chemistry class labels
# -------------------------------
def classify_material(row):
    if row["has_RE"] and row["has_3d_TM"]:
        return "RE–TM hybrid"
    elif row["has_RE"]:
        return "Rare-earth based"
    elif row["has_3d_TM"]:
        return "3d-transition-metal"
    else:
        return "Other"

df["chemistry_class"] = df.apply(classify_material, axis=1)

# -------------------------------
# 5. Save enriched chemistry table
# -------------------------------
chem_path = DATA_INTERIM_DIR / "Curie_Data_Chemistry_Parsed.csv"
df.to_csv(chem_path, index=False)

print("Chemistry-enriched dataset saved to:", chem_path)

# -------------------------------
# 6. Summary tables
# -------------------------------
class_counts = df["chemistry_class"].value_counts()
class_counts_path = TABLES_DIR / "chemistry_class_counts.csv"
class_counts.to_csv(class_counts_path)

print("\nMaterial count by chemistry class:")
print(class_counts)
print("Saved to:", class_counts_path)

# -------------------------------
# 7. Tc statistics by chemistry class
# -------------------------------
tc_by_class = df.groupby("chemistry_class")["Curie"].describe()
tc_by_class_path = TABLES_DIR / "Tc_by_chemistry_class.csv"
tc_by_class.to_csv(tc_by_class_path)

print("\nTc statistics by chemistry class saved to:", tc_by_class_path)


In [ ]:
# ============================================================
# CELL 4A: Tc vs CHEMISTRY CLASS (PUBLICATION FIGURES)
# ============================================================

# -------------------------------
# 1. Load chemistry-enriched dataset
# -------------------------------
df = pd.read_csv(DATA_INTERIM_DIR / "Curie_Data_Chemistry_Parsed.csv")

print("Dataset loaded for plotting. Shape:", df.shape)

# -------------------------------
# 2. Boxplot: Tc by chemistry class
# -------------------------------
fig1, ax1 = plt.subplots()

df.boxplot(
    column="Curie",
    by="chemistry_class",
    ax=ax1,
    grid=False
)

ax1.set_xlabel("Chemistry Class")
ax1.set_ylabel("Curie Temperature (K)")
ax1.set_title("Curie Temperature Distribution by Chemistry Class")
plt.suptitle("")  # remove automatic pandas title

save_figure(fig1, "Tc_by_chemistry_class_boxplot")
plt.close(fig1)

# -------------------------------
# 3. Violin plot: distribution shape
# -------------------------------
fig2, ax2 = plt.subplots()

sns.violinplot(
    data=df,
    x="chemistry_class",
    y="Curie",
    cut=0,
    inner="quartile",
    ax=ax2
)

ax2.set_xlabel("Chemistry Class")
ax2.set_ylabel("Curie Temperature (K)")
ax2.set_title("Curie Temperature Distribution (Violin Plot)")

save_figure(fig2, "Tc_by_chemistry_class_violin")
plt.close(fig2)

# -------------------------------
# 4. Fraction of high-Tc materials (>300 K)
# -------------------------------
high_tc_threshold = 300

high_tc_fraction = (
    df.assign(high_tc=df["Curie"] > high_tc_threshold)
      .groupby("chemistry_class")["high_tc"]
      .mean()
      .sort_values(ascending=False)
)

high_tc_fraction_path = TABLES_DIR / "high_Tc_fraction_by_class.csv"
high_tc_fraction.to_csv(high_tc_fraction_path)

print("\nFraction of materials with Tc >", high_tc_threshold, "K:")
print(high_tc_fraction)
print("Saved to:", high_tc_fraction_path)

print("\nCELL 4A completed successfully.")


In [ ]:
# ============================================================
# CELL X: SAFE COMPOSITION PARSING & FILTERING
# ============================================================

from pymatgen.core import Composition
import pandas as pd

# Load chemistry-parsed dataset
df = pd.read_csv(DATA_INTERIM_DIR / "Curie_Data_Chemistry_Parsed.csv")

valid_rows = []
invalid_rows = []

for idx, row in df.iterrows():
    formula = row["Material_Name"]
    try:
        comp = Composition(formula)
        valid_rows.append((idx, comp))
    except Exception:
        invalid_rows.append(formula)

# Build valid dataframe
valid_indices = [idx for idx, _ in valid_rows]
df_valid = df.loc[valid_indices].copy()

# Attach composition column
df_valid["composition"] = [comp for _, comp in valid_rows]

print("Original rows :", df.shape[0])
print("Valid formulas:", df_valid.shape[0])
print("Invalid formulas removed:", len(invalid_rows))

# Save invalid formulas (VERY IMPORTANT for SI)
invalid_path = RESULTS_DIR / "invalid_formula_entries.csv"
pd.Series(invalid_rows, name="Invalid_Material_Name").to_csv(
    invalid_path, index=False
)

print("Invalid formulas saved to:", invalid_path)

# Overwrite df for downstream cells
df = df_valid.copy()
# ------------------------------------------------
# Save valid-formula-only dataset (IMPORTANT)
# ------------------------------------------------
valid_path = DATA_INTERIM_DIR / "Curie_Data_Valid_Formulas.csv"
df_valid.to_csv(valid_path, index=False)

print("Valid-formula dataset saved to:", valid_path)


In [ ]:
# ============================================================
# CELL X: DEEP CHEMICAL SANITIZATION (STRICT)
# ============================================================

import pandas as pd
import re
from pymatgen.core import Element, Composition

# ------------------------------------------------------------
# 1. Load current "valid formulas" dataset
# ------------------------------------------------------------
df = pd.read_csv(DATA_INTERIM_DIR / "Curie_Data_Valid_Formulas.csv")

print("Input rows:", df.shape[0])

# ------------------------------------------------------------
# 2. Helper: check if formula is STRICTLY elemental
# ------------------------------------------------------------
def is_strict_formula(formula):
    """
    Returns (True, None) if formula contains ONLY real elements.
    Returns (False, reason) otherwise.
    """
    try:
        comp = Composition(formula)
    except Exception:
        return False, "Composition parse failed"

    for sym in comp.get_el_amt_dict():
        try:
            Element(sym)
        except Exception:
            return False, f"Non-element token: {sym}"

    return True, None


# ------------------------------------------------------------
# 3. Apply strict validation
# ------------------------------------------------------------
valid_rows = []
invalid_rows = []

for _, row in df.iterrows():
    formula = row["Material_Name"]
    ok, reason = is_strict_formula(formula)
    if ok:
        valid_rows.append(row)
    else:
        invalid_rows.append({
            "Material_Name": formula,
            "Reason": reason
        })

df_strict = pd.DataFrame(valid_rows)
df_invalid = pd.DataFrame(invalid_rows)

print("Strictly valid formulas :", df_strict.shape[0])
print("Rejected (chemically unsafe):", df_invalid.shape[0])

# ------------------------------------------------------------
# 4. Save outputs (IMPORTANT)
# ------------------------------------------------------------
strict_path = DATA_INTERIM_DIR / "Curie_Data_Strict_Formulas.csv"
invalid_path = RESULTS_DIR / "invalid_formula_entries_strict.csv"

df_strict.to_csv(strict_path, index=False)
df_invalid.to_csv(invalid_path, index=False)

print("\nSaved STRICT dataset to:", strict_path)
print("Saved rejected formulas to:", invalid_path)


In [ ]:
# ============================================================
# CELL 5: EXTENDED PHYSICS + MATMINER DESCRIPTORS
#        (STRICT, FINAL, OVERLAP-SAFE)
# ============================================================

import numpy as np
import pandas as pd

from pymatgen.core import Composition, Element
from matminer.featurizers.composition import (
    ElementProperty,
    Stoichiometry,
    ValenceOrbital,
    IonProperty,
    AtomicOrbitals,
    BandCenter
)

# ------------------------------------------------------------
# 1. Load STRICTLY CLEANED dataset
# ------------------------------------------------------------
df = pd.read_csv(DATA_INTERIM_DIR / "Curie_Data_Strict_Formulas.csv")
print("Dataset loaded (strict formulas only):", df.shape)

# Guaranteed safe
df["composition"] = df["Material_Name"].apply(Composition)

# ------------------------------------------------------------
# 2. Remove any existing physics columns (RESTART SAFE)
# ------------------------------------------------------------
PHYS_COLS = [
    "n_elements", "composition_entropy", "max_atomic_fraction",
    "frac_3d_elements", "frac_RE_elements",
    "Z_mean_w", "Z_std", "X_mean_w", "X_range",
    "R_mean_w", "R_range",
    "VEC_mean", "VEC_std",
    "has_Fe", "has_Co", "has_Mn", "has_RE"
]

df = df.drop(columns=[c for c in PHYS_COLS if c in df.columns], errors="ignore")

# ------------------------------------------------------------
# 3. PHYSICS-INFORMED HANDCRAFTED FEATURES
# ------------------------------------------------------------

MAGNETIC_3D = {"Fe", "Co", "Ni", "Mn", "Cr", "V"}
RARE_EARTHS = {
    "La","Ce","Pr","Nd","Sm","Eu","Gd","Tb","Dy",
    "Ho","Er","Tm","Yb","Lu"
}

def physics_features(comp):
    el_amt = comp.get_el_amt_dict()
    total = sum(el_amt.values())

    els = [Element(sym) for sym in el_amt]
    fracs = {Element(sym): el_amt[sym] / total for sym in el_amt}
    frac_vals = np.array(list(fracs.values()))

    Z = np.array([e.Z for e in els])
    X = np.array([e.X if e.X is not None else np.nan for e in els])
    R = np.array([e.atomic_radius if e.atomic_radius is not None else np.nan for e in els])
    group = np.array([e.group if e.group is not None else np.nan for e in els])

    entropy = -np.sum(frac_vals * np.log(frac_vals))

    return pd.Series({
        # Chemical complexity
        "n_elements": len(els),
        "composition_entropy": entropy,
        "max_atomic_fraction": frac_vals.max(),

        # Magnetic chemistry
        "frac_3d_elements": sum(fracs[e] for e in fracs if e.symbol in MAGNETIC_3D),
        "frac_RE_elements": sum(fracs[e] for e in fracs if e.symbol in RARE_EARTHS),

        # Atomic statistics (weighted)
        "Z_mean_w": np.nansum(Z * frac_vals),
        "Z_std": np.nanstd(Z),
        "X_mean_w": np.nansum(X * frac_vals),
        "X_range": np.nanmax(X) - np.nanmin(X),
        "R_mean_w": np.nansum(R * frac_vals),
        "R_range": np.nanmax(R) - np.nanmin(R),

        # Valence / Slater–Pauling proxies
        "VEC_mean": np.nansum(group * frac_vals),
        "VEC_std": np.nanstd(group),

        # Flags
        "has_Fe": int("Fe" in el_amt),
        "has_Co": int("Co" in el_amt),
        "has_Mn": int("Mn" in el_amt),
        "has_RE": int(any(Element(sym).symbol in RARE_EARTHS for sym in el_amt)),
    })

# SAFE concat (no column overlap issues)
df_phys = pd.concat(
    [df, df["composition"].apply(physics_features)],
    axis=1
)

print("Handcrafted physics-informed features added.")

# ------------------------------------------------------------
# 4. EXTENDED MATMINER FEATURES
# ------------------------------------------------------------

ep = ElementProperty.from_preset("magpie")
stoich = Stoichiometry()
valence = ValenceOrbital(props=["avg", "frac"])
ion = IonProperty(fast=True)
orbitals = AtomicOrbitals()
bandcenter = BandCenter()

df_feat = ep.featurize_dataframe(df_phys, "composition", ignore_errors=True)
df_feat = stoich.featurize_dataframe(df_feat, "composition", ignore_errors=True)
df_feat = valence.featurize_dataframe(df_feat, "composition", ignore_errors=True)
df_feat = ion.featurize_dataframe(df_feat, "composition", ignore_errors=True)
df_feat = orbitals.featurize_dataframe(df_feat, "composition", ignore_errors=True)
df_feat = bandcenter.featurize_dataframe(df_feat, "composition", ignore_errors=True)

print("Extended matminer features generated.")

# ------------------------------------------------------------
# 5. FINAL ML FEATURE TABLE
# ------------------------------------------------------------

DROP_COLS = [
    "Material_Name",
    "elements",
    "composition",
    "chemistry_class"
]

df_final = df_feat.drop(columns=DROP_COLS, errors="ignore")

feature_path = DATA_INTERIM_DIR / "Curie_Feature_Table_Magpie_Physics.csv"
df_final.to_csv(feature_path, index=False)

print("FINAL feature table saved to:", feature_path)
print("Final feature table shape:", df_final.shape)


In [ ]:
# ============================================================
# CELL 5B: FEATURE TABLE INSPECTION & SANITY CHECKS (FIXED)
# ============================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ------------------------------------------------------------
# 1. Load feature table
# ------------------------------------------------------------
df = pd.read_csv(DATA_INTERIM_DIR / "Curie_Feature_Table_Magpie_Physics.csv")

print("Feature table shape:", df.shape)

# ------------------------------------------------------------
# 2. Separate numeric features safely
# ------------------------------------------------------------
numeric_df = df.select_dtypes(include=[np.number])

print("Numeric feature count (excluding target):", numeric_df.shape[1] - 1)

# ------------------------------------------------------------
# 3. Target variable inspection
# ------------------------------------------------------------
print("\n=== Curie Temperature Statistics ===")
print(df["Curie"].describe())

df["Curie"].describe().to_csv(
    RESULTS_DIR / "Tc_target_statistics.csv"
)

# ------------------------------------------------------------
# 4. Missing value check (NUMERIC ONLY)
# ------------------------------------------------------------
missing_frac = numeric_df.isna().mean().sort_values(ascending=False)
missing_frac.to_csv(
    RESULTS_DIR / "feature_missing_fraction.csv"
)

print("\nTop numeric features with missing values:")
print(missing_frac[missing_frac > 0].head(10))

# ------------------------------------------------------------
# 5. Low-variance feature check (NUMERIC ONLY)
# ------------------------------------------------------------
numeric_features_only = numeric_df.drop(columns=["Curie"], errors="ignore")

variances = numeric_features_only.var().sort_values()
low_var = variances[variances < 1e-6]

low_var.to_csv(
    RESULTS_DIR / "low_variance_features.csv"
)

print("\nLow-variance numeric features (<1e-6):", len(low_var))

# ------------------------------------------------------------
# 6. Correlation with Tc (NUMERIC ONLY)
# ------------------------------------------------------------
corr_with_tc = numeric_features_only.corrwith(df["Curie"]).sort_values()

corr_with_tc.to_csv(
    RESULTS_DIR / "feature_correlation_with_Tc.csv"
)

print("\nTop + correlated features with Tc:")
print(corr_with_tc.tail(10))

print("\nTop − correlated features with Tc:")
print(corr_with_tc.head(10))

# ------------------------------------------------------------
# 7. Physics sanity plots
# ------------------------------------------------------------
fig, ax = plt.subplots()
ax.hist(df["Curie"], bins=50)
ax.set_xlabel("Curie Temperature (K)")
ax.set_ylabel("Count")
ax.set_title("Curie Temperature Distribution")
save_figure(fig, "Tc_distribution_feature_table")
plt.close(fig)

KEY_FEATURES = [
    "n_elements",
    "frac_3d_elements",
    "frac_RE_elements",
    "VEC_mean",
    "composition_entropy"
]

for feat in KEY_FEATURES:
    if feat in df.columns:
        fig, ax = plt.subplots()
        ax.scatter(df[feat], df["Curie"], s=5, alpha=0.3)
        ax.set_xlabel(feat)
        ax.set_ylabel("Curie Temperature (K)")
        ax.set_title(f"Tc vs {feat}")
        save_figure(fig, f"Tc_vs_{feat}")
        plt.close(fig)

# ------------------------------------------------------------
# 8. Summary
# ------------------------------------------------------------
summary = {
    "n_samples": df.shape[0],
    "n_numeric_features": numeric_features_only.shape[1],
    "n_features_with_nan": int((missing_frac > 0).sum()),
    "n_low_variance_features": int(len(low_var))
}

summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    RESULTS_DIR / "feature_table_summary.csv",
    index=False
)

print("\n=== FEATURE TABLE SUMMARY ===")
print(summary_df)

print("\nCELL 5B COMPLETED SUCCESSFULLY.")


In [ ]:
# ============================================================
# CELL 6: Tc REGRESSION — MULTI-MODEL BENCHMARKING (FINAL)
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    median_absolute_error,
    r2_score
)

from scipy.stats import pearsonr, spearmanr

import xgboost as xgb
import lightgbm as lgb

# ------------------------------------------------------------
# 1. Load feature table (numeric only)
# ------------------------------------------------------------
df = pd.read_csv(DATA_INTERIM_DIR / "Curie_Feature_Table_Magpie_Physics.csv")

y = df["Curie"].values
X = df.drop(columns=["Curie"])

# keep numeric features only (critical)
X = X.select_dtypes(include=[np.number])

print("Final ML matrix shape:", X.shape)

# ------------------------------------------------------------
# 2. Train / test split (REPRODUCIBLE)
# ------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# Save split indices
np.savez(
    RESULTS_DIR / "train_test_indices.npz",
    train_idx=X_train.index.values,
    test_idx=X_test.index.values
)

# ------------------------------------------------------------
# 3. Metric function (ranking-aware)
# ------------------------------------------------------------
def regression_metrics(y_true, y_pred):
    return {
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
        "R2": r2_score(y_true, y_pred),
        "MAPE_%": np.mean(
            np.abs((y_true - y_pred) / np.clip(y_true, 1e-6, None))
        ) * 100,
        "Pearson_r": pearsonr(y_true, y_pred)[0],
        "Spearman_r": spearmanr(y_true, y_pred)[0]
    }

# ------------------------------------------------------------
# 4. Define models (ALL)
# ------------------------------------------------------------
models = {
    "Ridge": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("model", Ridge(alpha=1.0))
    ]),

    "RandomForest": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("model", RandomForestRegressor(
            n_estimators=500,
            random_state=42,
            n_jobs=-1
        ))
    ]),

    "GradientBoosting": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("model", GradientBoostingRegressor(
            random_state=42
        ))
    ]),

    "XGBoost": xgb.XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    ),

    "LightGBM": lgb.LGBMRegressor(
        n_estimators=500,
        learning_rate=0.05,
        random_state=42,
        n_jobs=-1
    ),

    "MLP": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("model", MLPRegressor(
            hidden_layer_sizes=(256,128,64),
            activation="relu",
            max_iter=1000,
            random_state=42
        ))
    ])
}

# ------------------------------------------------------------
# 5. Train, evaluate, save EVERYTHING
# ------------------------------------------------------------
metrics_all = []

for name, model in models.items():
    print(f"\nTraining {name} ...")

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Metrics
    m = regression_metrics(y_test, y_pred)
    m["Model"] = name
    metrics_all.append(m)

    # Save model
    joblib.dump(
        model,
        RESULTS_DIR / f"model_{name}.joblib"
    )

    # Save predictions
    pred_df = pd.DataFrame({
        "y_true": y_test,
        "y_pred": y_pred,
        "residual": y_test - y_pred
    })
    pred_df.to_csv(
        RESULTS_DIR / f"predictions_{name}.csv",
        index=False
    )

    # -------- PLOTS --------
    # True vs Predicted
    fig, ax = plt.subplots()
    ax.scatter(y_test, y_pred, s=10, alpha=0.5)
    ax.plot(
        [y_test.min(), y_test.max()],
        [y_test.min(), y_test.max()],
        "--",
        color="black"
    )
    ax.set_xlabel("True Tc (K)")
    ax.set_ylabel("Predicted Tc (K)")
    ax.set_title(f"True vs Predicted Tc — {name}")
    save_figure(fig, f"Tc_true_vs_pred_{name}")
    plt.close(fig)

    # Residuals
    fig, ax = plt.subplots()
    ax.hist(y_test - y_pred, bins=50)
    ax.set_xlabel("Residual (K)")
    ax.set_ylabel("Count")
    ax.set_title(f"Residual Distribution — {name}")
    save_figure(fig, f"Tc_residuals_{name}")
    plt.close(fig)

# ------------------------------------------------------------
# 6. Consolidated metrics table
# ------------------------------------------------------------
metrics_df = pd.DataFrame(metrics_all).set_index("Model")
metrics_df = metrics_df.sort_values("RMSE")

metrics_df.to_csv(
    RESULTS_DIR / "regression_metrics_all_models.csv"
)

print("\n=== REGRESSION BENCHMARK RESULTS ===")
print(metrics_df)

print("\nSaved metrics to:",
      RESULTS_DIR / "regression_metrics_all_models.csv")

print("\nCELL 6 COMPLETED SUCCESSFULLY.")
# ------------------------------------------------------------
# 7. RADAR (SPIDER) PLOT — MODEL PERFORMANCE COMPARISON
# ------------------------------------------------------------

import numpy as np

# ---- Metrics to include in radar plot ----
# NOTE: RMSE, MAE, MedAE, MAPE should be minimized → invert later
RADAR_METRICS = [
    "RMSE",
    "MAE",
    "MedAE",
    "R2",
    "Pearson_r",
    "Spearman_r"
]

radar_df = metrics_df[RADAR_METRICS].copy()

# ---- Normalize metrics to [0,1] ----
# Higher is better for radar plot
radar_norm = pd.DataFrame(index=radar_df.index)

for col in radar_df.columns:
    if col in ["RMSE", "MAE", "MedAE"]:
        # lower is better → invert
        radar_norm[col] = 1 - (
            (radar_df[col] - radar_df[col].min()) /
            (radar_df[col].max() - radar_df[col].min())
        )
    else:
        # higher is better
        radar_norm[col] = (
            (radar_df[col] - radar_df[col].min()) /
            (radar_df[col].max() - radar_df[col].min())
        )

# Save normalized radar table (SI-ready)
radar_norm.to_csv(
    RESULTS_DIR / "radar_metrics_normalized.csv"
)

# ---- Radar plot setup ----
labels = radar_norm.columns.tolist()
num_vars = len(labels)

angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]  # close loop

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

for model in radar_norm.index:
    values = radar_norm.loc[model].tolist()
    values += values[:1]

    ax.plot(angles, values, linewidth=2, label=model)
    ax.fill(angles, values, alpha=0.10)

# ---- Formatting ----
ax.set_theta_offset(np.pi / 2)
ax.set_theta_direction(-1)

ax.set_thetagrids(
    np.degrees(angles[:-1]),
    labels
)

ax.set_ylim(0, 1)
ax.set_title("Model Performance Comparison (Radar Plot)", pad=20)
ax.legend(loc="upper right", bbox_to_anchor=(1.35, 1.15))

# ---- Save figure ----
save_figure(fig, "Tc_model_comparison_radar")
plt.close(fig)

print("Radar plot saved: Tc_model_comparison_radar (PNG + PDF)")



In [ ]:
import numpy as np, pandas as pd, shap, joblib, matplotlib.pyplot as plt

# ---- Paths
OUT = RESULTS_DIR / "shap" / "RF"
OUT.mkdir(parents=True, exist_ok=True)

# ---- Data
df = pd.read_csv(DATA_INTERIM_DIR / "Curie_Feature_Table_Magpie_Physics.csv")
X = df.drop(columns=["Curie"]).select_dtypes(include=[np.number])
feature_names = X.columns.tolist()

# ---- Subsample for speed (reproducible)
X_shap = X.sample(800, random_state=42)

# ---- Model
rf = joblib.load(RESULTS_DIR / "model_RandomForest.joblib")
if hasattr(rf, "named_steps"):
    rf = rf.named_steps["model"]

# ---- SHAP
explainer = shap.TreeExplainer(rf)
shap_vals = explainer.shap_values(X_shap)

mean_abs = np.abs(shap_vals).mean(axis=0)
rank = (pd.DataFrame({"feature": feature_names, "mean_abs_shap": mean_abs})
        .sort_values("mean_abs_shap", ascending=False))
rank.to_csv(OUT / "shap_mean_abs.csv", index=False)

# ---- Plots
fig = plt.figure()
shap.summary_plot(shap_vals, X_shap, max_display=25, show=False)
save_figure(fig, "RF_shap_summary", outdir=OUT); plt.close(fig)

fig = plt.figure()
shap.summary_plot(shap_vals, X_shap, plot_type="bar", max_display=25, show=False)
save_figure(fig, "RF_shap_bar", outdir=OUT); plt.close(fig)

print("RF SHAP done.")


In [ ]:
import numpy as np
import pandas as pd
import shap
import joblib
import matplotlib.pyplot as plt
from tqdm import tqdm

# -------------------------------
# Paths
# -------------------------------
OUT = RESULTS_DIR / "shap" / "RF"
OUT.mkdir(parents=True, exist_ok=True)

# -------------------------------
# Load data
# -------------------------------
df = pd.read_csv(DATA_INTERIM_DIR / "Curie_Feature_Table_Magpie_Physics.csv")
X = df.drop(columns=["Curie"]).select_dtypes(include=[np.number])
feature_names = X.columns.tolist()

# Subsample for SHAP (speed + reproducibility)
X_shap = X.sample(800, random_state=42)

# -------------------------------
# Load model
# -------------------------------
rf = joblib.load(RESULTS_DIR / "model_RandomForest.joblib")
if hasattr(rf, "named_steps"):
    rf = rf.named_steps["model"]

# -------------------------------
# SHAP computation
# -------------------------------
print("Running SHAP for RandomForest...")
explainer = shap.TreeExplainer(rf)
shap_vals = explainer.shap_values(X_shap)

# -------------------------------
# Save mean |SHAP|
# -------------------------------
mean_abs = np.abs(shap_vals).mean(axis=0)
rank = pd.DataFrame({
    "feature": feature_names,
    "mean_abs_shap": mean_abs
}).sort_values("mean_abs_shap", ascending=False)

rank.to_csv(OUT / "shap_mean_abs.csv", index=False)

# -------------------------------
# Plots
# -------------------------------
fig = plt.figure()
shap.summary_plot(shap_vals, X_shap, max_display=25, show=False)
fig.savefig(OUT / "RF_shap_summary.png", dpi=300, bbox_inches="tight")
fig.savefig(OUT / "RF_shap_summary.pdf", bbox_inches="tight")
plt.close(fig)

fig = plt.figure()
shap.summary_plot(shap_vals, X_shap, plot_type="bar", max_display=25, show=False)
fig.savefig(OUT / "RF_shap_bar.png", dpi=300, bbox_inches="tight")
fig.savefig(OUT / "RF_shap_bar.pdf", bbox_inches="tight")
plt.close(fig)

print("RF SHAP completed.")


In [ ]:
import numpy as np
import pandas as pd
import shap
import joblib
import matplotlib.pyplot as plt
from tqdm import tqdm

OUT = RESULTS_DIR / "shap" / "LGBM"
OUT.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(DATA_INTERIM_DIR / "Curie_Feature_Table_Magpie_Physics.csv")
X = df.drop(columns=["Curie"]).select_dtypes(include=[np.number])
feature_names = X.columns.tolist()

X_shap = X.sample(800, random_state=42)

lgbm = joblib.load(RESULTS_DIR / "model_LightGBM.joblib")

print("Running SHAP for LightGBM...")
explainer = shap.TreeExplainer(lgbm)
shap_vals = explainer.shap_values(X_shap)

mean_abs = np.abs(shap_vals).mean(axis=0)
rank = pd.DataFrame({
    "feature": feature_names,
    "mean_abs_shap": mean_abs
}).sort_values("mean_abs_shap", ascending=False)

rank.to_csv(OUT / "shap_mean_abs.csv", index=False)

fig = plt.figure()
shap.summary_plot(shap_vals, X_shap, max_display=25, show=False)
fig.savefig(OUT / "LGBM_shap_summary.png", dpi=300, bbox_inches="tight")
fig.savefig(OUT / "LGBM_shap_summary.pdf", bbox_inches="tight")
plt.close(fig)

fig = plt.figure()
shap.summary_plot(shap_vals, X_shap, plot_type="bar", max_display=25, show=False)
fig.savefig(OUT / "LGBM_shap_bar.png", dpi=300, bbox_inches="tight")
fig.savefig(OUT / "LGBM_shap_bar.pdf", bbox_inches="tight")
plt.close(fig)

print("LGBM SHAP completed.")


In [ ]:
import numpy as np
import pandas as pd
import shap
import joblib
import matplotlib.pyplot as plt
from tqdm import tqdm

OUT = RESULTS_DIR / "shap" / "XGB"
OUT.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(DATA_INTERIM_DIR / "Curie_Feature_Table_Magpie_Physics.csv")
X = df.drop(columns=["Curie"]).select_dtypes(include=[np.number])
feature_names = X.columns.tolist()

X_shap = X.sample(800, random_state=42)

xgb_model = joblib.load(RESULTS_DIR / "model_XGBoost.joblib")

def xgb_predict(Xi):
    return xgb_model.predict(Xi)

print("Running SHAP for XGBoost (callable mode)...")

masker = shap.maskers.Independent(X_shap)
explainer = shap.Explainer(xgb_predict, masker)

shap_exp = explainer(X_shap)
shap_vals = shap_exp.values

mean_abs = np.abs(shap_vals).mean(axis=0)
rank = pd.DataFrame({
    "feature": feature_names,
    "mean_abs_shap": mean_abs
}).sort_values("mean_abs_shap", ascending=False)

rank.to_csv(OUT / "shap_mean_abs.csv", index=False)

fig = plt.figure()
shap.summary_plot(shap_vals, X_shap, max_display=25, show=False)
fig.savefig(OUT / "XGB_shap_summary.png", dpi=300, bbox_inches="tight")
fig.savefig(OUT / "XGB_shap_summary.pdf", bbox_inches="tight")
plt.close(fig)

fig = plt.figure()
shap.summary_plot(shap_vals, X_shap, plot_type="bar", max_display=25, show=False)
fig.savefig(OUT / "XGB_shap_bar.png", dpi=300, bbox_inches="tight")
fig.savefig(OUT / "XGB_shap_bar.pdf", bbox_inches="tight")
plt.close(fig)

print("XGB SHAP completed.")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

BASE = RESULTS_DIR / "shap"
TABLE_OUT = RESULTS_DIR / "tables"
FIG_OUT = RESULTS_DIR / "figures"

models = ["RF", "LGBM", "XGB"]

tables = []
for m in models:
    t = pd.read_csv(BASE / m / "shap_mean_abs.csv").head(10)
    t["Model"] = m
    tables.append(t)

shap_all = pd.concat(tables, ignore_index=True)
shap_all.to_csv(TABLE_OUT / "shap_top10_all_models.csv", index=False)

# Consensus
consensus = (
    shap_all
    .pivot_table(index="feature", values="mean_abs_shap", aggfunc="mean")
    .sort_values("mean_abs_shap", ascending=False)
)
consensus.to_csv(TABLE_OUT / "shap_consensus.csv")

# Radar plot
rad = shap_all.pivot_table(
    index="Model", columns="feature", values="mean_abs_shap"
).fillna(0.0)

rad = rad.div(rad.max())

labels = rad.columns.tolist()
angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False).tolist()
angles += angles[:1]

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
for m in rad.index:
    vals = rad.loc[m].tolist()
    vals += vals[:1]
    ax.plot(angles, vals, linewidth=2, label=m)
    ax.fill(angles, vals, alpha=0.15)

ax.set_thetagrids(np.degrees(angles[:-1]), labels)
ax.set_title("SHAP Feature Importance — Model Comparison (Radar)", pad=20)
ax.legend(loc="upper right", bbox_to_anchor=(1.35, 1.15))

fig.savefig(FIG_OUT / "shap_radar_comparison.png", dpi=300, bbox_inches="tight")
fig.savefig(FIG_OUT / "shap_radar_comparison.pdf", bbox_inches="tight")
plt.close(fig)

print("Consensus SHAP + radar plot saved.")


In [ ]:
MODEL_NAME = "XGBoost"

import numpy as np, pandas as pd, matplotlib.pyplot as plt

# ------------------------------------------------------------
# Directories
# ------------------------------------------------------------
BASE_DIR = RESULTS_DIR / "error_analysis" / MODEL_NAME
TABLE_DIR = BASE_DIR / "tables"
FIG_DIR = BASE_DIR / "figures"
TABLE_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------------
# Load data
# ------------------------------------------------------------
df = pd.read_csv(DATA_INTERIM_DIR / "Curie_Feature_Table_Magpie_Physics.csv")

# Reconstruct chemistry_class
def classify_chemistry(row):
    if row["frac_RE_elements"] > 0.2 and row["frac_3d_elements"] > 0.2:
        return "RE–TM hybrid"
    elif row["frac_RE_elements"] > 0.2:
        return "Rare-earth based"
    elif row["frac_3d_elements"] > 0.2:
        return "3d-transition-metal"
    else:
        return "Other"

df["chemistry_class"] = df.apply(classify_chemistry, axis=1)

# Load split + predictions
idx = np.load(RESULTS_DIR / "train_test_indices.npz", allow_pickle=True)
test_idx = idx["test_idx"]
pred = pd.read_csv(RESULTS_DIR / "predictions_XGBoost.csv")

df_test = df.loc[test_idx].copy()
df_test["Tc_true"] = pred["y_true"].values
df_test["Tc_pred"] = pred["y_pred"].values
df_test["residual"] = df_test["Tc_true"] - df_test["Tc_pred"]
df_test["abs_error"] = np.abs(df_test["residual"])

# ------------------------------------------------------------
# Global residuals
# ------------------------------------------------------------
fig, ax = plt.subplots()
ax.hist(df_test["residual"], bins=60)
ax.set_xlabel("Residual (K)")
ax.set_ylabel("Count")
ax.set_title("Residual Distribution — XGBoost")
fig.savefig(FIG_DIR / "residual_distribution.png", dpi=300, bbox_inches="tight")
fig.savefig(FIG_DIR / "residual_distribution.pdf", bbox_inches="tight")
plt.close(fig)

df_test["abs_error"].describe().to_csv(TABLE_DIR / "global_error_statistics.csv")

# ------------------------------------------------------------
# Chemistry-resolved errors
# ------------------------------------------------------------
chem_err = (
    df_test.groupby("chemistry_class")["abs_error"]
    .agg(["mean", "median", "std", "count"])
)
chem_err.to_csv(TABLE_DIR / "error_by_chemistry_class.csv")

# ------------------------------------------------------------
# Physics-informed failure probes
# ------------------------------------------------------------
df_test.groupby(df_test["frac_RE_elements"] > 0.2)["abs_error"].mean().to_csv(
    TABLE_DIR / "error_RE_rich_vs_not.csv"
)

df_test.groupby(df_test["frac_3d_elements"] > 0.3)["abs_error"].mean().to_csv(
    TABLE_DIR / "error_3d_rich_vs_not.csv"
)

pd.qcut(
    df_test["composition_entropy"], 3,
    labels=["Low", "Medium", "High"]
).to_frame("entropy_bin").join(df_test).groupby(
    "entropy_bin"
)["abs_error"].mean().to_csv(
    TABLE_DIR / "error_by_entropy_regime.csv"
)

print(f"CELL 7A COMPLETED: {MODEL_NAME}")


In [ ]:
MODEL_NAME = "RandomForest"

import numpy as np, pandas as pd, matplotlib.pyplot as plt

BASE_DIR = RESULTS_DIR / "error_analysis" / MODEL_NAME
TABLE_DIR = BASE_DIR / "tables"
FIG_DIR = BASE_DIR / "figures"
TABLE_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(DATA_INTERIM_DIR / "Curie_Feature_Table_Magpie_Physics.csv")

def classify_chemistry(row):
    if row["frac_RE_elements"] > 0.2 and row["frac_3d_elements"] > 0.2:
        return "RE–TM hybrid"
    elif row["frac_RE_elements"] > 0.2:
        return "Rare-earth based"
    elif row["frac_3d_elements"] > 0.2:
        return "3d-transition-metal"
    else:
        return "Other"

df["chemistry_class"] = df.apply(classify_chemistry, axis=1)

idx = np.load(RESULTS_DIR / "train_test_indices.npz", allow_pickle=True)
test_idx = idx["test_idx"]
pred = pd.read_csv(RESULTS_DIR / "predictions_RandomForest.csv")

df_test = df.loc[test_idx].copy()
df_test["Tc_true"] = pred["y_true"].values
df_test["Tc_pred"] = pred["y_pred"].values
df_test["residual"] = df_test["Tc_true"] - df_test["Tc_pred"]
df_test["abs_error"] = np.abs(df_test["residual"])

df_test["abs_error"].describe().to_csv(TABLE_DIR / "global_error_statistics.csv")

chem_err = df_test.groupby("chemistry_class")["abs_error"].mean()
chem_err.to_csv(TABLE_DIR / "error_by_chemistry_class.csv")

print(f"CELL 7B COMPLETED: {MODEL_NAME}")


In [ ]:
MODEL_NAME = "LightGBM"

import numpy as np, pandas as pd, matplotlib.pyplot as plt

BASE_DIR = RESULTS_DIR / "error_analysis" / MODEL_NAME
TABLE_DIR = BASE_DIR / "tables"
FIG_DIR = BASE_DIR / "figures"
TABLE_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(DATA_INTERIM_DIR / "Curie_Feature_Table_Magpie_Physics.csv")

def classify_chemistry(row):
    if row["frac_RE_elements"] > 0.2 and row["frac_3d_elements"] > 0.2:
        return "RE–TM hybrid"
    elif row["frac_RE_elements"] > 0.2:
        return "Rare-earth based"
    elif row["frac_3d_elements"] > 0.2:
        return "3d-transition-metal"
    else:
        return "Other"

df["chemistry_class"] = df.apply(classify_chemistry, axis=1)

idx = np.load(RESULTS_DIR / "train_test_indices.npz", allow_pickle=True)
test_idx = idx["test_idx"]
pred = pd.read_csv(RESULTS_DIR / "predictions_LightGBM.csv")

df_test = df.loc[test_idx].copy()
df_test["Tc_true"] = pred["y_true"].values
df_test["Tc_pred"] = pred["y_pred"].values
df_test["residual"] = df_test["Tc_true"] - df_test["Tc_pred"]
df_test["abs_error"] = np.abs(df_test["residual"])

df_test["abs_error"].describe().to_csv(TABLE_DIR / "global_error_statistics.csv")

chem_err = df_test.groupby("chemistry_class")["abs_error"].mean()
chem_err.to_csv(TABLE_DIR / "error_by_chemistry_class.csv")

print(f"CELL 7C COMPLETED: {MODEL_NAME}")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ------------------------------------------------------------
# 0. Paths
# ------------------------------------------------------------
BASE_ERR = RESULTS_DIR / "error_analysis"
TABLE_DIR = BASE_ERR / "tables_cross_model"
FIG_DIR = BASE_ERR / "figures_cross_model"

TABLE_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

MODELS = ["XGBoost", "RandomForest", "LightGBM"]

# ------------------------------------------------------------
# 1. LOAD GLOBAL ERROR STATISTICS (ROBUST)
# ------------------------------------------------------------
global_stats = []

for m in MODELS:
    stats = pd.read_csv(
        BASE_ERR / m / "tables" / "global_error_statistics.csv",
        index_col=0
    )

    # stats is a single-column DataFrame from Series.describe()
    series = stats.iloc[:, 0]

    row = {
        "Model": m,
        "MAE_mean": series.loc["mean"],
        "MAE_median": series.loc["50%"],
        "MAE_std": series.loc["std"],
        "MAE_90p": series.quantile(0.90)
    }

    global_stats.append(row)

global_df = pd.DataFrame(global_stats).set_index("Model")
global_df.to_csv(TABLE_DIR / "global_error_comparison.csv")

print("Global error comparison:")
print(global_df)


# ------------------------------------------------------------
# 2. CHEMISTRY-RESOLVED ERROR COMPARISON
# ------------------------------------------------------------
chem_tables = []

for m in MODELS:
    chem = pd.read_csv(
        BASE_ERR / m / "tables" / "error_by_chemistry_class.csv",
        index_col=0
    )
    chem["Model"] = m
    chem_tables.append(chem.reset_index())

chem_df = pd.concat(chem_tables)
chem_df.to_csv(TABLE_DIR / "error_by_chemistry_cross_model.csv", index=False)

# Pivot for comparison table
chem_pivot = chem_df.pivot_table(
    index="chemistry_class",
    columns="Model",
    values="mean"
)
chem_pivot.to_csv(TABLE_DIR / "error_by_chemistry_pivot.csv")

# ------------------------------------------------------------
# 3. ERROR REGIME COMPARISON (Tc REGIMES)
# ------------------------------------------------------------
regime_tables = []

for m in MODELS:
    path = BASE_ERR / m / "tables" / "error_by_Tc_regime.csv"
    if path.exists():
        reg = pd.read_csv(path)
        reg["Model"] = m
        regime_tables.append(reg)

if regime_tables:
    regime_df = pd.concat(regime_tables)
    regime_df.to_csv(TABLE_DIR / "error_by_Tc_regime_cross_model.csv", index=False)

# ------------------------------------------------------------
# 4. RADAR (SPIDER) PLOT — GLOBAL ERROR METRICS
# ------------------------------------------------------------
# Normalize: lower error = better → invert scale
radar_df = global_df.copy()

for col in radar_df.columns:
    radar_df[col] = 1 - (
        (radar_df[col] - radar_df[col].min()) /
        (radar_df[col].max() - radar_df[col].min())
    )

labels = radar_df.columns.tolist()
angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
angles += angles[:1]

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

for model in radar_df.index:
    values = radar_df.loc[model].tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=2, label=model)
    ax.fill(angles, values, alpha=0.15)

ax.set_thetagrids(np.degrees(angles[:-1]), labels)
ax.set_ylim(0, 1)
ax.set_title("Cross-Model Error Performance Comparison (Radar)", pad=20)
ax.legend(loc="upper right", bbox_to_anchor=(1.35, 1.15))

fig.savefig(FIG_DIR / "error_radar_global.png", dpi=300, bbox_inches="tight")
fig.savefig(FIG_DIR / "error_radar_global.pdf", bbox_inches="tight")
plt.close(fig)

# ------------------------------------------------------------
# 5. RADAR — CHEMISTRY-SPECIFIC PERFORMANCE (OPTIONAL BUT VALUABLE)
# ------------------------------------------------------------
chem_norm = chem_pivot.copy()

for col in chem_norm.columns:
    chem_norm[col] = 1 - (
        (chem_norm[col] - chem_norm[col].min()) /
        (chem_norm[col].max() - chem_norm[col].min())
    )

labels = chem_norm.index.tolist()
angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
angles += angles[:1]

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

for model in chem_norm.columns:
    vals = chem_norm[model].tolist()
    vals += vals[:1]
    ax.plot(angles, vals, linewidth=2, label=model)
    ax.fill(angles, vals, alpha=0.15)

ax.set_thetagrids(np.degrees(angles[:-1]), labels)
ax.set_ylim(0, 1)
ax.set_title("Model Robustness Across Chemistry Classes (Radar)", pad=20)
ax.legend(loc="upper right", bbox_to_anchor=(1.35, 1.15))

fig.savefig(FIG_DIR / "error_radar_chemistry.png", dpi=300, bbox_inches="tight")
fig.savefig(FIG_DIR / "error_radar_chemistry.pdf", bbox_inches="tight")
plt.close(fig)

print("\nCELL 7D COMPLETED SUCCESSFULLY.")
print("Cross-model comparison tables & radar plots saved to:")
print(BASE_ERR)


In [ ]:
import numpy as np
import pandas as pd
from itertools import combinations
from pymatgen.core import Composition

# ------------------------------------------------------------
# 0. Paths
# ------------------------------------------------------------
INV_DIR = RESULTS_DIR / "inverse_design"
TABLE_DIR = INV_DIR / "tables"
INV_DIR.mkdir(exist_ok=True, parents=True)
TABLE_DIR.mkdir(exist_ok=True, parents=True)

# ------------------------------------------------------------
# 1. ELEMENT POOLS (DATA-INFORMED)
# ------------------------------------------------------------
TM_3D = ["Fe", "Co", "Mn", "Ni", "Cr", "V"]
RARE_EARTHS = ["Gd", "Tb", "Dy", "Ho", "Er"]
MAIN_GROUP = ["Al", "Si", "Ge", "Ga", "Sn"]

ELEMENT_POOL = TM_3D + RARE_EARTHS + MAIN_GROUP

# ------------------------------------------------------------
# 2. STOICHIOMETRIC PATTERNS
# ------------------------------------------------------------
STOICH_PATTERNS = {
    2: [(0.5, 0.5)],
    3: [(0.33, 0.33, 0.34)],
    4: [(0.25, 0.25, 0.25, 0.25)]
}

# ------------------------------------------------------------
# 3. PHYSICS-BASED CONSTRAINTS (FIXED)
# ------------------------------------------------------------
def physics_constraints(comp: Composition):
    comp_frac = comp.fractional_composition
    fracs = comp_frac.get_el_amt_dict()
    els = list(fracs.keys())

    frac_3d = sum(fracs.get(e, 0) for e in TM_3D)
    frac_re = sum(fracs.get(e, 0) for e in RARE_EARTHS)

    entropy = -sum(v * np.log(v) for v in fracs.values())

    if frac_3d < 0.30:
        return False
    if frac_re > 0.40:
        return False
    if entropy > 1.35:
        return False
    if len(els) > 4:
        return False

    return True

# ------------------------------------------------------------
# 4. GENERATE CANDIDATES
# ------------------------------------------------------------
candidates = []

for n_elem, patterns in STOICH_PATTERNS.items():
    for elems in combinations(ELEMENT_POOL, n_elem):
        for frac_pattern in patterns:
            comp_dict = dict(zip(elems, frac_pattern))
            comp = Composition(comp_dict)
            candidates.append({
                "formula": comp.reduced_formula,
                "composition": comp
            })

df_candidates = pd.DataFrame(candidates).drop_duplicates("formula")

df_candidates.to_csv(
    TABLE_DIR / "inverse_candidates_raw.csv",
    index=False
)

print("Raw inverse candidates:", len(df_candidates))

# ------------------------------------------------------------
# 5. APPLY PHYSICS CONSTRAINTS
# ------------------------------------------------------------
mask = df_candidates["composition"].apply(physics_constraints)
df_filtered = df_candidates[mask].copy()

df_filtered["n_elements"] = df_filtered["composition"].apply(
    lambda c: len(c.fractional_composition.get_el_amt_dict())
)

df_filtered["composition_entropy"] = df_filtered["composition"].apply(
    lambda c: -sum(
        v * np.log(v)
        for v in c.fractional_composition.get_el_amt_dict().values()
    )
)

df_filtered["frac_3d_elements"] = df_filtered["composition"].apply(
    lambda c: sum(
        c.fractional_composition.get_el_amt_dict().get(e, 0)
        for e in TM_3D
    )
)

df_filtered["frac_RE_elements"] = df_filtered["composition"].apply(
    lambda c: sum(
        c.fractional_composition.get_el_amt_dict().get(e, 0)
        for e in RARE_EARTHS
    )
)

# ------------------------------------------------------------
# 6. SAVE FILTERED POOL
# ------------------------------------------------------------
df_filtered.drop(columns=["composition"]).to_csv(
    TABLE_DIR / "inverse_candidates_filtered.csv",
    index=False
)

# ------------------------------------------------------------
# 7. GENERATION LOG (METHODS-READY)
# ------------------------------------------------------------
log = pd.DataFrame({
    "Stage": ["Raw generation", "After physics constraints"],
    "Count": [len(df_candidates), len(df_filtered)]
})

log.to_csv(
    TABLE_DIR / "inverse_generation_log.csv",
    index=False
)

print("Filtered inverse candidates:", len(df_filtered))
print("CELL 8A COMPLETED SUCCESSFULLY.")


In [ ]:
import numpy as np
import pandas as pd
import joblib
from pymatgen.core import Composition

from matminer.featurizers.composition import (
    ElementProperty,
    Stoichiometry,
    ValenceOrbital,
    IonProperty,
    AtomicOrbitals,
    BandCenter
)

# ------------------------------------------------------------
# 0. Paths
# ------------------------------------------------------------
INV_DIR = RESULTS_DIR / "inverse_design"
TABLE_DIR = INV_DIR / "tables"
MODEL_DIR = RESULTS_DIR

TABLE_DIR.mkdir(exist_ok=True, parents=True)

# ------------------------------------------------------------
# 1. Load inverse candidates (from Cell 8A)
# ------------------------------------------------------------
df_inv = pd.read_csv(TABLE_DIR / "inverse_candidates_filtered.csv")
print("Inverse candidate pool:", df_inv.shape)

# ------------------------------------------------------------
# 2. Load TRAINING feature template (CRITICAL)
# ------------------------------------------------------------
df_train_feat = pd.read_csv(
    DATA_INTERIM_DIR / "Curie_Feature_Table_Magpie_Physics.csv"
)

FEATURE_COLS = (
    df_train_feat
    .drop(columns=["Curie"])
    .select_dtypes(include=[np.number])
    .columns
    .tolist()
)

# ------------------------------------------------------------
# 3. Recreate composition object
# ------------------------------------------------------------
df_inv["composition"] = df_inv["formula"].apply(Composition)

# ------------------------------------------------------------
# 4. MATMINER FEATURES ONLY (physics already exists)
# ------------------------------------------------------------
ep = ElementProperty.from_preset("magpie")
sto = Stoichiometry()
val = ValenceOrbital(props=["avg", "frac"])
ion = IonProperty(fast=True)
orb = AtomicOrbitals()
bc = BandCenter()

df_inv = ep.featurize_dataframe(df_inv, "composition", ignore_errors=True)
df_inv = sto.featurize_dataframe(df_inv, "composition", ignore_errors=True)
df_inv = val.featurize_dataframe(df_inv, "composition", ignore_errors=True)
df_inv = ion.featurize_dataframe(df_inv, "composition", ignore_errors=True)
df_inv = orb.featurize_dataframe(df_inv, "composition", ignore_errors=True)
df_inv = bc.featurize_dataframe(df_inv, "composition", ignore_errors=True)

# ------------------------------------------------------------
# 5. Align inverse feature matrix to training features
# ------------------------------------------------------------
X_inv = (
    df_inv
    .select_dtypes(include=[np.number])
    .reindex(columns=FEATURE_COLS)
)

print("Inverse feature matrix shape:", X_inv.shape)

# ------------------------------------------------------------
# 6. Load trained models (ALL THREE)
# ------------------------------------------------------------
models = {
    "XGBoost": joblib.load(MODEL_DIR / "model_XGBoost.joblib"),
    "RandomForest": joblib.load(MODEL_DIR / "model_RandomForest.joblib"),
    "LightGBM": joblib.load(MODEL_DIR / "model_LightGBM.joblib"),
}

# ------------------------------------------------------------
# 7. Predict Tc (SEPARATELY FOR EACH MODEL)
# ------------------------------------------------------------
pred_tables = []

for name, model in models.items():
    print(f"Predicting Tc using {name} ...")

    Tc_pred = model.predict(X_inv)

    df_pred = pd.DataFrame({
        "formula": df_inv["formula"],
        f"Tc_{name}": Tc_pred
    }).sort_values(f"Tc_{name}", ascending=False)

    df_pred.to_csv(
        TABLE_DIR / f"inverse_predictions_{name}.csv",
        index=False
    )

    pred_tables.append(df_pred.set_index("formula"))

# ------------------------------------------------------------
# 8. Merge predictions across models
# ------------------------------------------------------------
df_all = pd.concat(pred_tables, axis=1).reset_index()

df_all.to_csv(
    TABLE_DIR / "inverse_predictions_all_models.csv",
    index=False
)

# ------------------------------------------------------------
# 9. Summary statistics (high-level sanity check)
# ------------------------------------------------------------
summary = pd.DataFrame({
    "Model": list(models.keys()),
    "Max_Tc": [df_all[f"Tc_{m}"].max() for m in models],
    "Mean_Top10": [
        df_all[f"Tc_{m}"].nlargest(10).mean() for m in models
    ]
})

summary.to_csv(
    TABLE_DIR / "inverse_prediction_summary.csv",
    index=False
)

print("\nInverse prediction summary:")
print(summary)

print("\nCELL 8B COMPLETED SUCCESSFULLY.")


In [ ]:
import numpy as np
import pandas as pd

# ------------------------------------------------------------
# 0. Paths
# ------------------------------------------------------------
INV_DIR = RESULTS_DIR / "inverse_design"
TABLE_DIR = INV_DIR / "tables"
TABLE_DIR.mkdir(exist_ok=True, parents=True)

# ------------------------------------------------------------
# 1. Load inverse prediction tables
# ------------------------------------------------------------
df_all = pd.read_csv(TABLE_DIR / "inverse_predictions_all_models.csv")
df_base = pd.read_csv(TABLE_DIR / "inverse_candidates_filtered.csv")

# Merge physics info
df = df_all.merge(
    df_base,
    on="formula",
    how="left"
)

# ------------------------------------------------------------
# 2. MODEL AGREEMENT METRICS
# ------------------------------------------------------------
Tc_cols = ["Tc_XGBoost", "Tc_RandomForest", "Tc_LightGBM"]

df["Tc_mean"] = df[Tc_cols].mean(axis=1)
df["Tc_std"] = df[Tc_cols].std(axis=1)
df["Tc_min"] = df[Tc_cols].min(axis=1)

# Agreement score: high mean, low disagreement
df["agreement_score"] = df["Tc_mean"] / (1 + df["Tc_std"])

# ------------------------------------------------------------
# 3. ERROR-AWARE RELIABILITY PENALTY
# ------------------------------------------------------------
# From Cell 7: ~1600 K = extreme MAE tail
ERROR_SCALE = 1600.0

df["reliability_score"] = np.exp(
    -df["Tc_std"] / ERROR_SCALE
)

# ------------------------------------------------------------
# 4. PHYSICS CONSISTENCY SCORE
# ------------------------------------------------------------
df["physics_score"] = (
    0.5 * df["frac_3d_elements"]
    - 0.3 * df["frac_RE_elements"]
    - 0.2 * df["composition_entropy"]
)

# Normalize physics score
df["physics_score"] = (
    df["physics_score"] - df["physics_score"].min()
) / (
    df["physics_score"].max() - df["physics_score"].min()
)

# ------------------------------------------------------------
# 5. FINAL INVERSE SCORE
# ------------------------------------------------------------
df["inverse_score"] = (
    0.5 * df["agreement_score"].rank(pct=True)
    + 0.3 * df["reliability_score"]
    + 0.2 * df["physics_score"]
)

df = df.sort_values("inverse_score", ascending=False)

# ------------------------------------------------------------
# 6. SAVE FULL RANKED LIST
# ------------------------------------------------------------
df.to_csv(
    TABLE_DIR / "inverse_candidates_scored.csv",
    index=False
)

# ------------------------------------------------------------
# 7. TOP CANDIDATES (PUBLICATION-GRADE)
# ------------------------------------------------------------
TOP_N = 50
df_top = df.head(TOP_N)

df_top.to_csv(
    TABLE_DIR / "inverse_candidates_top50.csv",
    index=False
)

# ------------------------------------------------------------
# 8. AGREEMENT SUMMARY (METHODS-READY)
# ------------------------------------------------------------
summary = pd.DataFrame({
    "Metric": [
        "Candidates generated",
        "After physics constraints",
        "After model agreement",
        "Final shortlisted"
    ],
    "Count": [
        len(df_all),
        len(df),
        len(df[df["Tc_std"] < 300]),
        TOP_N
    ]
})

summary.to_csv(
    TABLE_DIR / "inverse_candidate_statistics.csv",
    index=False
)

# ------------------------------------------------------------
# 9. MODEL AGREEMENT TABLE
# ------------------------------------------------------------
agreement = df_top[[
    "formula",
    "Tc_XGBoost",
    "Tc_RandomForest",
    "Tc_LightGBM",
    "Tc_mean",
    "Tc_std",
    "inverse_score"
]]

agreement.to_csv(
    TABLE_DIR / "inverse_agreement_summary.csv",
    index=False
)

print("\nTOP 10 INVERSE CANDIDATES:")
print(agreement.head(10))

print("\nCELL 8C COMPLETED SUCCESSFULLY.")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# ------------------------------------------------------------
# 0. Paths
# ------------------------------------------------------------
INV_DIR = RESULTS_DIR / "inverse_design"
TABLE_DIR = INV_DIR / "tables"
FIG_DIR = INV_DIR / "figures"

FIG_DIR.mkdir(exist_ok=True, parents=True)

# ------------------------------------------------------------
# 1. Load FINAL ranked inverse candidates (from Cell 8C)
# ------------------------------------------------------------
df = pd.read_csv(
    TABLE_DIR / "inverse_candidates_scored.csv"
)

print("Inverse candidates loaded:", df.shape)

# ------------------------------------------------------------
# 2. Global summary statistics
# ------------------------------------------------------------
summary = df[[
    "Tc_mean",
    "Tc_std",
    "inverse_score",
    "frac_3d_elements",
    "frac_RE_elements",
    "composition_entropy"
]].describe()

summary.to_csv(
    TABLE_DIR / "inverse_physics_summary_statistics.csv"
)

# ------------------------------------------------------------
# 3. Tc vs 3d-element fraction
# ------------------------------------------------------------
fig, ax = plt.subplots()
ax.scatter(
    df["frac_3d_elements"],
    df["Tc_mean"],
    s=25,
    alpha=0.6
)
ax.set_xlabel("Fraction of 3d transition metals")
ax.set_ylabel("Mean predicted Tc (K)")
ax.set_title("Tc vs 3d-element fraction (inverse candidates)")
save_figure(fig, "Tc_vs_frac_3d_inverse")
plt.close(fig)

# ------------------------------------------------------------
# 4. Tc vs rare-earth fraction
# ------------------------------------------------------------
fig, ax = plt.subplots()
ax.scatter(
    df["frac_RE_elements"],
    df["Tc_mean"],
    s=25,
    alpha=0.6,
    color="darkred"
)
ax.set_xlabel("Fraction of rare-earth elements")
ax.set_ylabel("Mean predicted Tc (K)")
ax.set_title("Tc vs rare-earth fraction (inverse candidates)")
save_figure(fig, "Tc_vs_frac_RE_inverse")
plt.close(fig)

# ------------------------------------------------------------
# 5. Tc vs compositional entropy
# ------------------------------------------------------------
fig, ax = plt.subplots()
ax.scatter(
    df["composition_entropy"],
    df["Tc_mean"],
    s=25,
    alpha=0.6,
    color="darkgreen"
)
ax.set_xlabel("Composition entropy")
ax.set_ylabel("Mean predicted Tc (K)")
ax.set_title("Tc vs entropy (inverse candidates)")
save_figure(fig, "Tc_vs_entropy_inverse")
plt.close(fig)

# ------------------------------------------------------------
# 6. Inverse score vs Tc uncertainty
# ------------------------------------------------------------
fig, ax = plt.subplots()
ax.scatter(
    df["Tc_std"],
    df["inverse_score"],
    s=25,
    alpha=0.6,
    color="purple"
)
ax.set_xlabel("Model disagreement (Tc std, K)")
ax.set_ylabel("Inverse score")
ax.set_title("Inverse score vs model uncertainty")
save_figure(fig, "Inverse_score_vs_Tc_std")
plt.close(fig)

# ------------------------------------------------------------
# 7. Top candidates chemistry table (publication-ready)
# ------------------------------------------------------------
TOP_N = 20

chem_table = df.head(TOP_N)[[
    "formula",
    "Tc_XGBoost",
    "Tc_RandomForest",
    "Tc_LightGBM",
    "Tc_mean",
    "Tc_std",
    "frac_3d_elements",
    "frac_RE_elements",
    "composition_entropy",
    "inverse_score"
]]

chem_table.to_csv(
    TABLE_DIR / "top20_inverse_candidates_physics.csv",
    index=False
)

print("\nTop inverse candidates (physics view):")
print(chem_table.head(10))

print("\nCELL 8D COMPLETED SUCCESSFULLY.")


In [ ]:
# ============================================================
# CELL 9A: SPINTRONIC PROXY DESCRIPTORS (CLEAN & ROBUST)
# ============================================================

import numpy as np
import pandas as pd

# ------------------------------------------------------------
# 0. Paths (CONSISTENT WITH NOTEBOOK)
# ------------------------------------------------------------
INV_DIR = RESULTS_DIR / "inverse_design"
TABLE_DIR = INV_DIR / "tables"

TABLE_DIR.mkdir(exist_ok=True, parents=True)

# ------------------------------------------------------------
# 1. Load inverse candidates with physics + Tc (from Cell 8C)
# ------------------------------------------------------------
df = pd.read_csv(TABLE_DIR / "inverse_candidates_scored.csv")

print("Inverse candidates loaded:", df.shape)

# ------------------------------------------------------------
# 2. SAFETY CHECK: REQUIRED COLUMNS
# ------------------------------------------------------------
required_cols = [
    "frac_3d_elements",
    "frac_RE_elements",
    "composition_entropy",
    "inverse_score",
    "Tc_mean"
]

missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise KeyError(f"Missing required columns: {missing}")

# Optional Magpie columns (may or may not exist)
HAS_GAP = "MagpieData mean GSbandgap" in df.columns
HAS_COVDEV = "MagpieData avg_dev CovalentRadius" in df.columns

# ------------------------------------------------------------
# 3. SPINTRONIC PROXY DEFINITIONS (NO DFT)
# ------------------------------------------------------------
def spintronic_proxies(row):
    frac_3d = row["frac_3d_elements"]
    frac_re = row["frac_RE_elements"]
    entropy = row["composition_entropy"]

    # ---- Exchange network strength ----
    exchange_proxy = frac_3d * (1.0 - frac_re)

    # ---- Spin polarization tendency ----
    spin_polarization_proxy = frac_3d / (1.0 + entropy)

    # ---- Metallic transport proxy ----
    if HAS_GAP:
        gap = row["MagpieData mean GSbandgap"]
        metallicity_proxy = np.exp(-gap)
    else:
        metallicity_proxy = 1.0  # assume metallic if unknown

    # ---- Disorder / scattering proxy ----
    if HAS_COVDEV:
        cov_dev = row["MagpieData avg_dev CovalentRadius"]
        scattering_proxy = np.exp(-cov_dev)
    else:
        scattering_proxy = 1.0  # no penalty if unknown

    # ---- Composite spintronics score ----
    spintronics_score = (
        0.35 * exchange_proxy +
        0.25 * spin_polarization_proxy +
        0.20 * metallicity_proxy +
        0.20 * scattering_proxy
    )

    return pd.Series({
        "exchange_proxy": exchange_proxy,
        "spin_polarization_proxy": spin_polarization_proxy,
        "metallicity_proxy": metallicity_proxy,
        "scattering_proxy": scattering_proxy,
        "spintronics_score": spintronics_score
    })

# ------------------------------------------------------------
# 4. APPLY SPINTRONIC PROXIES
# ------------------------------------------------------------
df_spin = df.join(df.apply(spintronic_proxies, axis=1))

print("Spintronic proxy features added.")

# ------------------------------------------------------------
# 5. NORMALIZE & FINAL DEVICE SCORE
# ------------------------------------------------------------
df_spin["spintronics_score_norm"] = (
    df_spin["spintronics_score"] - df_spin["spintronics_score"].min()
) / (
    df_spin["spintronics_score"].max()
    - df_spin["spintronics_score"].min()
)

# Device-oriented final score
df_spin["final_device_score"] = (
    0.6 * df_spin["inverse_score"] +
    0.4 * df_spin["spintronics_score_norm"]
)

df_spin = df_spin.sort_values(
    "final_device_score", ascending=False
)

# ------------------------------------------------------------
# 6. SAVE RESULTS (FULL + TOP)
# ------------------------------------------------------------
df_spin.to_csv(
    TABLE_DIR / "inverse_candidates_with_spintronics.csv",
    index=False
)

TOP_N = 50
df_spin.head(TOP_N).to_csv(
    TABLE_DIR / "inverse_candidates_spintronics_top50.csv",
    index=False
)

# ------------------------------------------------------------
# 7. METHODS-READY SUMMARY
# ------------------------------------------------------------
summary = pd.DataFrame({
    "Stage": [
        "Inverse candidates (physics filtered)",
        "Spintronics evaluated",
        "Final shortlisted"
    ],
    "Count": [
        len(df),
        len(df_spin),
        TOP_N
    ]
})

summary.to_csv(
    TABLE_DIR / "spintronics_screening_summary.csv",
    index=False
)

# ------------------------------------------------------------
# 8. QUICK SANITY PRINT
# ------------------------------------------------------------
print("\nTOP 10 SPINTRONICALLY PROMISING CANDIDATES:")
print(
    df_spin[[
        "formula",
        "Tc_mean",
        "exchange_proxy",
        "spin_polarization_proxy",
        "metallicity_proxy",
        "scattering_proxy",
        "spintronics_score",
        "final_device_score"
    ]].head(10)
)

print("\nCELL 9A COMPLETED SUCCESSFULLY.")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ------------------------------------------------------------
# 0. Paths
# ------------------------------------------------------------
INV_DIR = RESULTS_DIR / "inverse_design"
TABLE_DIR = INV_DIR / "tables"
FIG_DIR_PNG = RESULTS_DIR / "figures" / "png"
FIG_DIR_PDF = RESULTS_DIR / "figures" / "pdf"

FIG_DIR_PNG.mkdir(exist_ok=True, parents=True)
FIG_DIR_PDF.mkdir(exist_ok=True, parents=True)

# ------------------------------------------------------------
# 1. Load spintronics-ranked inverse candidates (from Cell 9A)
# ------------------------------------------------------------
df = pd.read_csv(TABLE_DIR / "inverse_candidates_with_spintronics.csv")

print("Loaded spintronics-evaluated candidates:", df.shape)

# ------------------------------------------------------------
# 2. Select TOP candidates for radar visualization
# ------------------------------------------------------------
TOP_K = 5
df_top = df.head(TOP_K).copy()

# Metrics to visualize (all normalized or bounded)
RADAR_FEATURES = [
    "Tc_mean",
    "exchange_proxy",
    "spin_polarization_proxy",
    "metallicity_proxy",
    "scattering_proxy",
    "inverse_score"
]

# ------------------------------------------------------------
# 3. Normalize features for radar (0–1)
# ------------------------------------------------------------
df_radar = df_top[["formula"] + RADAR_FEATURES].copy()

for col in RADAR_FEATURES:
    min_v = df[col].min()
    max_v = df[col].max()
    if max_v > min_v:
        df_radar[col] = (df_radar[col] - min_v) / (max_v - min_v)
    else:
        df_radar[col] = 0.5  # fallback

# ------------------------------------------------------------
# 4. Radar plot function
# ------------------------------------------------------------
def radar_plot(df_radar, features, title, fname):
    labels = features
    num_vars = len(labels)

    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(
        figsize=(7, 7),
        subplot_kw=dict(polar=True)
    )

    for _, row in df_radar.iterrows():
        values = row[features].tolist()
        values += values[:1]

        ax.plot(angles, values, linewidth=2, label=row["formula"])
        ax.fill(angles, values, alpha=0.15)

    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)

    ax.set_thetagrids(
        np.degrees(angles[:-1]),
        labels
    )

    ax.set_ylim(0, 1)
    ax.set_title(title, fontsize=14, pad=20)
    ax.legend(
        loc="upper right",
        bbox_to_anchor=(1.35, 1.15),
        fontsize=9
    )

    # Save
    fig.savefig(FIG_DIR_PNG / f"{fname}.png", dpi=300, bbox_inches="tight")
    fig.savefig(FIG_DIR_PDF / f"{fname}.pdf", bbox_inches="tight")
    plt.close(fig)

# ------------------------------------------------------------
# 5. Generate radar plot
# ------------------------------------------------------------
radar_plot(
    df_radar,
    RADAR_FEATURES,
    title="Spintronic Signature Radar — Top Inverse Candidates",
    fname="spintronics_radar_top5"
)

print("Saved radar plots:")
print(" -", FIG_DIR_PNG / "spintronics_radar_top5.png")
print(" -", FIG_DIR_PDF / "spintronics_radar_top5.pdf")

print("\nCELL 9B COMPLETED SUCCESSFULLY.")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ------------------------------------------------------------
# 0. Paths
# ------------------------------------------------------------
INV_DIR = RESULTS_DIR / "inverse_design"
TABLE_DIR = INV_DIR / "tables"
FIG_PNG = RESULTS_DIR / "figures" / "png"
FIG_PDF = RESULTS_DIR / "figures" / "pdf"

FIG_PNG.mkdir(exist_ok=True, parents=True)
FIG_PDF.mkdir(exist_ok=True, parents=True)

# ------------------------------------------------------------
# 1. Load spintronics-ranked candidates
# ------------------------------------------------------------
df = pd.read_csv(TABLE_DIR / "inverse_candidates_with_spintronics.csv")

print("Spintronics-evaluated dataset:", df.shape)

# ------------------------------------------------------------
# 2. DEFINE CHEMISTRY CLASSES (RULE-BASED, TRANSPARENT)
# ------------------------------------------------------------
def chemistry_family(formula):
    if any(x in formula for x in ["Fe", "Co", "Ni"]):
        return "Fe–Co–Ni based"
    if "Mn" in formula:
        return "Mn-based"
    if any(x in formula for x in ["Cr", "V"]):
        return "Cr/V-based"
    return "Other"

df["chemistry_family"] = df["formula"].apply(chemistry_family)

# Binary vs ternary vs quaternary
df["complexity"] = df["formula"].str.count(r"[A-Z]")  # proxy

def complexity_class(n):
    if n == 2:
        return "Binary"
    if n == 3:
        return "Ternary"
    return "Higher-order"

df["complexity_class"] = df["complexity"].apply(complexity_class)

# ------------------------------------------------------------
# 3. AGGREGATED STATISTICS BY CHEMISTRY FAMILY
# ------------------------------------------------------------
chem_stats = (
    df.groupby("chemistry_family")
    .agg(
        count=("formula", "count"),
        Tc_mean=("Tc_mean", "mean"),
        Tc_std=("Tc_std", "mean"),
        spintronics_score=("spintronics_score", "mean"),
        final_device_score=("final_device_score", "mean"),
        entropy=("composition_entropy", "mean")
    )
    .sort_values("final_device_score", ascending=False)
)

chem_stats.to_csv(
    TABLE_DIR / "spintronics_by_chemistry_family.csv"
)

print("\nCHEMISTRY FAMILY SUMMARY:")
print(chem_stats)

# ------------------------------------------------------------
# 4. AGGREGATED STATISTICS BY CHEMICAL COMPLEXITY
# ------------------------------------------------------------
comp_stats = (
    df.groupby("complexity_class")
    .agg(
        count=("formula", "count"),
        Tc_mean=("Tc_mean", "mean"),
        Tc_std=("Tc_std", "mean"),
        spintronics_score=("spintronics_score", "mean"),
        final_device_score=("final_device_score", "mean"),
        entropy=("composition_entropy", "mean")
    )
    .sort_values("final_device_score", ascending=False)
)

comp_stats.to_csv(
    TABLE_DIR / "spintronics_by_complexity.csv"
)

print("\nCHEMICAL COMPLEXITY SUMMARY:")
print(comp_stats)

# ------------------------------------------------------------
# 5. VISUALIZATION — CHEMISTRY FAMILY COMPARISON
# ------------------------------------------------------------
fig, ax = plt.subplots(figsize=(7, 5))
chem_stats["final_device_score"].plot(
    kind="bar",
    ax=ax
)
ax.set_ylabel("Mean final device score")
ax.set_title("Spintronic performance by chemistry family")
ax.grid(axis="y", alpha=0.3)

fig.savefig(FIG_PNG / "spintronics_by_chemistry_family.png", dpi=300, bbox_inches="tight")
fig.savefig(FIG_PDF / "spintronics_by_chemistry_family.pdf", bbox_inches="tight")
plt.close(fig)

# ------------------------------------------------------------
# 6. VISUALIZATION — COMPLEXITY TRADE-OFF
# ------------------------------------------------------------
fig, ax = plt.subplots(figsize=(7, 5))
comp_stats["final_device_score"].plot(
    kind="bar",
    ax=ax,
    color="tab:orange"
)
ax.set_ylabel("Mean final device score")
ax.set_title("Spintronic performance vs chemical complexity")
ax.grid(axis="y", alpha=0.3)

fig.savefig(FIG_PNG / "spintronics_by_complexity.png", dpi=300, bbox_inches="tight")
fig.savefig(FIG_PDF / "spintronics_by_complexity.pdf", bbox_inches="tight")
plt.close(fig)

# ------------------------------------------------------------
# 7. DESIGN-RULE TABLE (DISCUSSION-READY)
# ------------------------------------------------------------
design_rules = pd.DataFrame({
    "Observation": [
        "Fe–Co–Ni alloys dominate top-ranked spintronic candidates",
        "Binary systems outperform ternary systems on average",
        "Higher entropy correlates with reduced device score",
        "Mn-based alloys show moderate Tc but higher disorder penalty"
    ],
    "Implication": [
        "Itinerant 3d exchange is the primary driver of spintronic performance",
        "Low chemical complexity favors coherent spin transport",
        "Entropy reduction is critical for reliable spintronics",
        "Mn systems may require structural ordering or substrates"
    ]
})

design_rules.to_csv(
    TABLE_DIR / "spintronics_design_rules.csv",
    index=False
)

print("\nDESIGN RULES TABLE GENERATED.")

print("\nCELL 9C COMPLETED SUCCESSFULLY.")
